<a href="https://colab.research.google.com/github/E-CG/AI4ENG/blob/master/02-%20Preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📦 Módulos, archivos, paquetes importantes para la ejecución del notebook.**

In [ ]:
! pip install py7zr

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
# Aquí debes cambiar la dirección donde tengas tus credenciales de Kaggle
! cp /content/drive/MyDrive/Modelos_I/credentials_kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download favorita-grocery-sales-forecasting
! unzip favorita-grocery-sales-forecasting.zip

In [ ]:
# Librerias uso básico
import numpy as np
import pandas as pd
import math as m
import time
import py7zr
import os
from subprocess import check_output

# Librerias preprocesado
from mlxtend.preprocessing import minmax_scaling

# Librerias para gráficar
import seaborn as sns
import matplotlib.pyplot as plt

# Funciones de sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,Ridge
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error

# **🗂️Leyendo y extrayendo los archivos .csv**

In [ ]:
# Ruta al archivo 7z en Google Drive
sevenzip_file_path = '/content'

# Directorio de destino para la extracción
extracted_dir = '/content/extracted_data/'

# Crear el directorio de destino si no existe
os.makedirs(extracted_dir, exist_ok=True)

# Listar los archivos .7z en el directorio de entrada
files_to_extract = [file for file in os.listdir(sevenzip_file_path) if file.endswith('.7z')]

# Iterar a través de los archivos y descomprimirlos
for file_to_extract in files_to_extract:
    with py7zr.SevenZipFile(os.path.join(sevenzip_file_path, file_to_extract), mode='r') as z:
        z.extractall(path=extracted_dir)

In [ ]:
test = pd.read_csv('/content/extracted_data/test.csv', usecols=[1, 2, 3, 4], parse_dates=["date"])
stores = pd.read_csv('/content/extracted_data/stores.csv')
items = pd.read_csv('/content/extracted_data/items.csv')
holidays_e = pd.read_csv('/content/extracted_data/holidays_events.csv', parse_dates=["date"])
transactions = pd.read_csv('/content/extracted_data/transactions.csv', parse_dates=["date"])
oil = pd.read_csv('/content/extracted_data/oil.csv')
print('Archivos cargados 🗞️✅')

Archivos cargados 🗞️✅


# **❌¿Qué archivos tienen datos nulos?**

🖊️ Se toma cada columna del dataframe en cuestion, se suman la cantidad de registros NaN y se convierte en un **porcentaje**. A cada columna se le saca tal porcentaje.

In [ ]:
oil_nan = (oil.isnull().sum() / oil.shape[0]) * 100
oil_nan

date          0.000000
dcoilwtico    3.530378
dtype: float64

Hay un 3.5% de datos faltantes en el archivo (oil.csv)

In [ ]:
store_nan = (stores.isnull().sum() / stores.shape[0]) * 100
store_nan

store_nbr    0.0
city         0.0
state        0.0
type         0.0
cluster      0.0
dtype: float64

No hay datos faltantes en (stores.csv)

In [ ]:
item_nan = (items.isnull().sum() / items.shape[0]) * 100
item_nan

item_nbr      0.0
family        0.0
class         0.0
perishable    0.0
dtype: float64

No hay datos faltantes en (items.csv)

In [ ]:
holiday_nan = (holidays_e.isnull().sum() / holidays_e.shape[0]) * 100
holiday_nan

In [ ]:
tran_nan = (transactions.isnull().sum() / transactions.shape[0]) * 100
tran_nan

# **🦾Cargando y leyendo archivo de entramiento**

In [ ]:
# Cargar el archivo CSV en chunks
chunked_dfs = pd.read_csv("/content/extracted_data/train.csv",
                          chunksize=20000,
                          usecols=[1, 2, 3, 4, 5],
                          parse_dates=['date'],
                          low_memory=False)

# Inicializar una lista para almacenar los DataFrames filtrados
datos_filtrados = []

# Iterar a través de los chunks
for chunk in chunked_dfs:
    # Crear una máscara booleana para las fechas entre 2015-10-01 y finales de 2017
    mask = (chunk['date'] >= '2015-10-01') & (chunk['date'] <= '2017-12-31')

    # Aplicar la máscara y agregar las filas filtradas a la lista
    trozo_filtrado = chunk[mask]
    datos_filtrados.append(trozo_filtrado)

# Concatenar los DataFrames filtrados en uno solo
train = pd.concat(datos_filtrados, ignore_index=True)

In [ ]:
train_nan = (train.isnull().sum() / train.shape[0]) * 100
train_nan

date           0.0
store_nbr      0.0
item_nbr       0.0
unit_sales     0.0
onpromotion    0.0
dtype: float64

No hay datos nulos en el dataframe de entrenamiento

# **📄Juntando los demás archivos en train.csv**

In [ ]:
train = pd.merge(train, stores, on='store_nbr')
train = pd.merge(train, items, on='store_nbr')
train = pd.merge(train, holiday_e, on='store_nbr')
train = pd.merge(train, store, on='store_nbr')

In [ ]:
train['onpromotion'] = train['onpromotion'].replace(True,1)
train['onpromotion'] = train['onpromotion'].replace(False,0)

Promedio del día anterior y día después. (Pensar si va a hacer por tienda o producto) ¿Qué voy a predicir? ¿venta unitaria de productos por tienda en x fecha? ¿Qué datos voy a usar? El laboratorio 3.2 da una visión de lo que podría hacer para rellenar los valores nulos.